In [1]:
import numpy as np
import torch
torch.set_default_dtype(torch.float64)
import matplotlib.pyplot as plt

In [ ]:
#Setting Network Parameters
num_pairs = 1
num_RX_ant = 2
num_TX_ant = 4
num_scatterers = torch.tensor([[2]])
Lam = 0.01
Ant_dist = 0.005
#Setting Locations
TX_locs = [torch.tensor([0.0, 0.0], dtype=torch.float64)]
RX_locs = [torch.tensor([-10.0, -10.0], dtype=torch.float64)]
SC_locs = np.array([[torch.tensor([[5.0, -5.0], [10, -5]], dtype=torch.float64)]])

#Obtaining Bt, Br, and A for each pair
class pair():
    def __init__(self, RX_idx, TX_idx):
        self.TX_idx = TX_idx
        self.RX_idx = RX_idx
        self.T_locs = [[i*Ant_dist+TX_locs[TX_idx][0], TX_locs[TX_idx][1]]  for i in range(num_TX_ant)]
        self.R_locs = [[i*Ant_dist+RX_locs[RX_idx][0], RX_locs[RX_idx][1]]  for i in range(num_RX_ant)]
        self.S_locs = [i for i in torch.from_numpy(SC_locs[RX_idx, TX_idx])]

    def calculate_Bt(self):
        Bt = torch.zeros(num_TX_ant, num_scatterers[self.RX_idx, self.TX_idx]+1, dtype=torch.complex64)
        # qT = (RX_locs[self.RX_idx] - TX_locs[self.TX_idx])
        qT = (RX_locs[self.RX_idx] - TX_locs[self.TX_idx])/torch.norm((RX_locs[self.RX_idx] - TX_locs[self.TX_idx]))
        D = torch.tensor(self.T_locs) - torch.tile(TX_locs[self.TX_idx], (num_TX_ant, 1))
        Bt[:, 0] = torch.exp(((-2*torch.pi*1j)/Lam)*(torch.matmul(qT, torch.transpose(D, 0, 1))))
        for i, S_loc in enumerate(self.S_locs):
            # qT = (S_loc - TX_locs[self.TX_idx])
            qT = (S_loc - TX_locs[self.TX_idx])/torch.norm((S_loc - TX_locs[self.TX_idx]))
            Bt[:, i+1] = torch.exp(((-2*torch.pi*1j)/Lam)*(torch.matmul(qT, torch.transpose(D, 0, 1))))
        return Bt

    def calculate_Br(self):
        Br = torch.zeros(num_RX_ant, num_scatterers[self.RX_idx, self.TX_idx]+1, dtype=torch.complex64)
        # qR = (RX_locs[self.RX_idx] - TX_locs[self.TX_idx])
        qR = (RX_locs[self.RX_idx] - TX_locs[self.TX_idx])/torch.norm((RX_locs[self.RX_idx] - TX_locs[self.TX_idx]))
        D = torch.tensor(self.R_locs) - torch.tile(RX_locs[self.RX_idx], (num_RX_ant, 1))
        Br[:, 0] = torch.exp(((2*torch.pi*1j)/Lam)*(torch.matmul(qR, torch.transpose(D, 0, 1))))
        for i, S_loc in enumerate(self.S_locs):
            # qR = (S_loc - RX_locs[self.RX_idx])
            qR = (RX_locs[self.RX_idx] - S_loc)/torch.norm((S_loc - RX_locs[self.RX_idx]))
            Br[:, i+1] = torch.exp(((2*torch.pi*1j)/Lam)*(torch.matmul(qR, torch.transpose(D, 0, 1))))
        return Br

    def calculate_A(self, L):
        A = torch.zeros(len(self.S_locs)+1, len(self.S_locs)+1, dtype=torch.complex64)
        r = torch.norm((RX_locs[self.RX_idx] - TX_locs[self.TX_idx]))
        A[0, 0] = (torch.exp(-2*torch.pi*(r/Lam)*1j))/r
        for i, S_loc in enumerate(self.S_locs):
            r = torch.norm((S_loc - TX_locs[self.TX_idx])) + torch.norm((RX_locs[self.RX_idx] - S_loc))
            # rand_ph = torch.rand(1) * 2 * torch.pi
            # rand_ph = torch.tensor(torch.pi/6)
            rand_ph = L[i]
            A[i+1, i+1] = ((torch.exp(-2*torch.pi*(r/Lam)*1j))*(torch.exp(rand_ph*1j)))/r
        return A
CHANGE OF p2

# Number of iterations
num_steps = 10001

# Store the first right eigenvector's entries over iterations
entries = torch.zeros((num_steps, 2), dtype=torch.complex64)
d = 2*torch.pi/100
p1_range = torch.arange(0, 2*torch.pi, d)
p2_range = torch.arange(0, 2*torch.pi, d)
step = 0
for p1 in p1_range:
    for p2 in p2_range:
        step+=1
        pair1 = pair(0, 0)
        B_R = pair1.calculate_Br()
        B_T = pair1.calculate_Br()
        A = pair1.calculate_A([torch.tensor(p1), torch.tensor(p2)])
        U2, S2, Vh2 = torch.linalg.svd(B_R @ A @ (B_T.conj().T))
        first_right_eigenvector = Vh2.conj().T[:, 0]

        # Store the entries of the first right eigenvector
        entries[step] = first_right_eigenvector

plt.figure(figsize=(10, 6))

for i in range(4):
    if i == 0:
        # First entry in blue
        plt.plot(range(num_steps), entries[:, i].real, label=f'Real Part Entry {i+1} (Blue)', color='blue', linestyle='-')
        plt.plot(range(num_steps), entries[:, i].imag, label=f'Imaginary Part Entry {i+1} (Blue)', color='blue', linestyle='--')
    else:
        # Other entries in red
        plt.plot(range(num_steps), entries[:, i].real, label=f'Real Part Entry {i+1} (Red)', color='red', linestyle='-')
        plt.plot(range(num_steps), entries[:, i].imag, label=f'Imaginary Part Entry {i+1} (Red)', color='red', linestyle='--')

plt.title("Entries of the First Right Eigenvector Over 100 Steps")
plt.xlabel("Step")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.show()

import torch
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Number of iterations
num_steps = 10001

# Store the first right eigenvector's entries over iterations
entries = torch.zeros((num_steps, 2), dtype=torch.complex64)
d = 2 * torch.pi / 100
p1_range = torch.arange(0, 2 * torch.pi, d)
p2_range = torch.arange(0, 2 * torch.pi, d)
step = 0
for p1 in p1_range:
    for p2 in p2_range:
        step += 1
        pair1 = pair(0, 0)
        B_R = pair1.calculate_Br()
        B_T = pair1.calculate_Br()
        A = pair1.calculate_A([torch.tensor(p1), torch.tensor(p2)])
        U2, S2, Vh2 = torch.linalg.svd(B_R @ A @ (B_T.conj().T))
        first_right_eigenvector = Vh2.conj().T[:, 0]

        # Store the entries of the first right eigenvector
        entries[step - 1] = first_right_eigenvector

# Create a figure for plotting
fig, ax = plt.subplots(figsize=(10, 6))
scat_blue = ax.scatter([], [], color="blue", label="Entry 1")
scat_red = ax.scatter([], [], color="red", label="Entry 2")
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_title("Entries of the First Right Eigenvector Over Steps")
ax.set_xlabel("Real Part")
ax.set_ylabel("Imaginary Part")
ax.legend()
ax.grid(True)

# Update function for animation
def update(frame):
    x_blue, y_blue = entries[frame, 0].real, entries[frame, 0].imag
    x_red, y_red = entries[frame, 1].real, entries[frame, 1].imag
    scat_blue.set_offsets([[x_blue, y_blue]])
    scat_red.set_offsets([[x_red, y_red]])
    return scat_blue, scat_red

# Create animation
ani = animation.FuncAnimation(
    fig, update, frames=num_steps, interval=10, blit=True
)

# Save as a video
ani.save("eigenvector_plot.mp4", writer="ffmpeg")

# Show the plot
plt.show()

U2, S2, Vh2 = torch.linalg.svd(B_R @ A @ (B_T.conj().T))
Vh2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Example: Two lists of 2x1 vectors with complex values
# list1 = [
#     np.array([[.1 + .2j], [.3 + .4j]]),
#     np.array([[.5 + .6j], [.7 + .8j]]),
#     np.array([[.9 + 1.0j], [1.1 + 1.2j]])
# ]

# list2 = [
#     np.array([[.2 + .1j], [.4 + .3j]]),
#     np.array([[.6 + .5j], [.8 + .7j]]),
#     np.array([[1.0 + .9j], [1.2 + 1.1j]])
# ]

list1 = u_1
list2 = u_2

# Function to format a single vector as a column vector string
def format_vector(vector):
    return "[\n" + "\n".join(f" {v[0].real:.2f} + {v[0].imag:.2f}j" for v in vector) + "\n]"

# Function to extract real and imaginary parts
def extract_real_imag(vector_list):
    real_parts = []
    imag_parts = []
    for vector in vector_list:
        for entry in vector:
            real_parts.append(entry[0].real)
            imag_parts.append(entry[0].imag)
    return real_parts, imag_parts

# Initialize the figure and subplots
fig, axes = plt.subplots(2, 1, figsize=(8, 10))

# Set up axes limits and labels for both plots
for ax in axes:
    ax.axhline(0, color='black', linewidth=0.5, linestyle='--')
    ax.axvline(0, color='black', linewidth=0.5, linestyle='--')
    ax.set_xlim(-1.2, 1.2)
    ax.set_ylim(-1.1, 2.3)
    ax.grid(True)
    ax.set_xlabel('Real Part')
    ax.set_ylabel('Imaginary Part')

axes[0].set_title('U[:, 0] Visualisation')
axes[1].set_title('U[:, 1] Visualisation')

# Colors for points (cycling for variety)
colors = ['blue', 'orange', 'green', 'purple', 'red']

# Animation update function
def update(frame):
    # Clear axes for each frame
    for ax in axes:
        ax.cla()
        ax.axhline(0, color='black', linewidth=0.5, linestyle='--')
        ax.axvline(0, color='black', linewidth=0.5, linestyle='--')
        ax.set_xlim(-1.2, 1.2)
        ax.set_ylim(-1.1, 2.3)
        ax.grid(True)
        ax.set_xlabel('Real Part')
        ax.set_ylabel('Imaginary Part')

    axes[0].set_title('U[:, 0] Visualisation')
    axes[1].set_title('U[:, 1] Visualisation')

    # Plot all vectors up to the current frame
    for i in range(frame + 1):
        real1, imag1 = extract_real_imag([list1[i]])
        real2, imag2 = extract_real_imag([list2[i]])

        # Plot vectors in consistent colors for both points
        axes[0].scatter(real1, imag1, color=colors[i % len(colors)], s=3)
        axes[1].scatter(real2, imag2, color=colors[i % len(colors)], s=3)

    # Get the current and previous vector values
    if frame > 0:
        list1_prev_text = f"U[:, 0]_Previous:\n{format_vector(list1[frame - 1])}"
        list2_prev_text = f"U[:, 0]_Previous:\n{format_vector(list2[frame - 1])}"
    else:
        list1_prev_text = "U[:, 0]_Previous:\n[ None ]"
        list2_prev_text = "U[:, 0]_Previous:\n[ None ]"

    list1_current_text = f"U[:, 0]_Current:\n{format_vector(list1[frame])}"
    list2_current_text = f"U[:, 1]_Current:\n{format_vector(list2[frame])}"

    # Display the current and previous vectors in the corner
    axes[0].text(-.5, 2, list1_prev_text, fontsize=8, color='black', ha='left', va='top', bbox=dict(facecolor='white', alpha=0.8))
    axes[0].text(0, 2, list1_current_text, fontsize=8, color='black', ha='left', va='top', bbox=dict(facecolor='white', alpha=0.8))

    axes[1].text(-.5, 2, list2_prev_text, fontsize=8, color='black', ha='left', va='top', bbox=dict(facecolor='white', alpha=0.8))
    axes[1].text(0, 2, list2_current_text, fontsize=8, color='black', ha='left', va='top', bbox=dict(facecolor='white', alpha=0.8))

# Create the animation
num_frames = len(list1)
ani = FuncAnimation(fig, update, frames=num_frames, interval=1000, repeat=False)

# Save animation to HTML with playback controls
ani.save('consistent_colors_vectors.html', writer='html')
print("Animation saved as 'consistent_colors_vectors.html'. Open it in a browser to see playback controls.")

# Display animation
plt.tight_layout()
plt.show()